In [3]:
import simfin as sf
import pandas as pd

In [ ]:
# Set your SimFin+ API-key for downloading data.
sf.set_api_key('')

# Set the local directory where data-files are stored.
# The directory will be created if it does not already exist.
sf.set_data_dir('~/simfin_data/')

# Download the data from the SimFin server and load into a Pandas DataFrame.
df_derived = sf.load_derived(variant='annual', market='us')
df_derived.to_csv('simfin_derived.csv')

In [ ]:
# Can be correlated to company IndustryIDs

sf.set_data_dir('~/simfin_data/')

# Download the data from the SimFin server and load into a Pandas DataFrame.
df_sector = sf.load_industries()

df_sector.to_csv('simfin_sectors.csv')

In [ ]:
# SimFin company IDs

sf.set_data_dir('~/simfin_data/')

# Download the data from the SimFin server and load into a Pandas DataFrame.
df_company = sf.load_companies(market='us')

df_company.to_csv('simfin_companies.csv')

In [2]:
# Share Price Ratios
sf.set_api_key('')
sf.set_data_dir('~/simfin_data/')

# Download the data from the SimFin server and load into a Pandas DataFrame.
df_price_ratios = sf.load_derived_shareprices(variant='daily', market='us')

df_price_ratios.to_csv('simfin_price_ratios.csv')

Dataset "us-derived-shareprices-daily" on disk (3 days old).
- Loading from disk ... Done!


### Prepare Price Ratio Data:

In [7]:
df_price_new = pd.read_csv('simfin_price_ratios.csv')

In [8]:
temp = df_price_new[['Symbol','Time','SimFinId','Market-Cap','Price to Earnings Ratio (ttm)','Price to Book Value','Book to Market Value']]

In [1]:
# Manually edit index headers "Ticker" and "Date" and save CSV file. 
# Alternatively, this will convert to single-level dataframe (where 'data' is the dataframe name):
# See: https://datascience.stackexchange.com/questions/55811/getting-stock-data-in-a-discipline-manner-from-yahoo-finance
# data.stack().reset_index().rename(index=str, columns={"level_1": "Symbol"}).sort_values(['Symbol','Date'])
# Then import:
# df_price_new = pd.read_csv('simfin_price_ratios.csv')

In [2]:
# Select useful columns:
temp = df_price_new[['Symbol','Time','SimFinId','Market-Cap',
                     'Price to Earnings Ratio (ttm)',
                     'Price to Book Value','Book to Market Value']]

In [31]:
# Select for only the last report of the year:
df_price_ratios2 = temp[temp['Time'].str.contains("2022-01-28")]

In [32]:
df_price_ratios2.shape

(2406, 7)

### Prepare Derived Data:

In [33]:
df_derived = pd.read_csv('simfin_derived.csv')

In [13]:
# df_derived = df_derived[['SimFinId','Fiscal Year','Current Ratio','Liabilities to Equity Ratio','Debt Ratio','Dividend Payout Ratio','Earnings Per Share, Diluted','Dividends Per Share','Piotroski F-Score']]

### Merge dataframes:

In [34]:
df_company = pd.read_csv('simfin_companies.csv')

In [35]:
# Merge dataframes (you can only merge two at a time)
newdf = pd.merge(df_company, df_derived, on='SimFinId')

In [36]:
newdf.shape

(23505, 35)

In [37]:
newdf = pd.merge(newdf, df_price_ratios2, on='SimFinId')

In [38]:
newdf = newdf.rename(columns={'Time': 'Report Date'})

In [39]:
newdf.to_csv('Current_Data.csv')

In [40]:
# Import pre-build CSV:
newdf = pd.read_csv('Current_Data.csv')

In [41]:
# convert float to object where necessary:
newdf["IndustryId"] = newdf.IndustryId.astype(object)
newdf["SimFinId"] = newdf.SimFinId.astype(object)

In [50]:
# Profile: Liabilities to Equity
df_temp = newdf[newdf['Report Date'].str.contains("2012-12-31")]
# metrics:
df_temp = df_temp[df_temp['Piotroski F-Score'].between(7, 9)
                  #& df_temp['Price to Earnings Ratio (ttm)'].between(-6000, -75)
                 # & df_temp['Price to Book Value'].between(-500, -3)
                 # & df_temp['Book to Market Value'].between(2, 300)
                 # & df_temp['Dividend Payout Ratio'].between(9, 15)
                 # & df_temp['Earnings Per Share, Diluted'].between(15, 50)
                 # & df_temp['Current Ratio'].between(8, 1000)
                  & df_temp['Liabilities to Equity Ratio'].between(-500, .5)
                 # & df_temp['Debt Ratio'].between(-30, .01)
                 ]
# Drop multiple rows belonging to a single stock:
df_temp = df_temp.drop_duplicates(['Symbol'])
df_temp.shape
df_temp['Symbol'].to_csv('Jan_30_2022_symbols_L2E.csv', index=False)